<a href="https://colab.research.google.com/github/Samin-Sadaf7/Book-of-LLMs/blob/main/HandsOnLLM_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 MB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
Failed to build llama-cpp-python
ERROR: ERROR: Failed to build installable wheels

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

#Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    )
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#Create a pipeline
pipe =pipeline(
    "text-generation",
    model = model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda


In [3]:
#Prompt
messages = [
    {
        "role":"user",
        "content":"Create a funny joke about chickens."
    }
]
#Generate the output
output = pipe(messages)
print(output[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Why did the chicken join the band? Because it had the drumsticks!


In [4]:
#Apply Prompt template
prompt = pipe.tokenizer.apply_chat_template(
    messages,
    tokenize=False
)
print(prompt)

<|user|>
Create a funny joke about chickens.<|end|>
<|endoftext|>


In [5]:
#Using high temperature
for _ in range(10):
  output = pipe(messages, do_sample=True, temperature=1)
  print(output[0]["generated_text"])

 Why did the chickens fight over the spilled milk?
 Once upon a time, there was a chicken that loved to tell jokes. One day, she got on a bus, and when she got off, a person yelled, "Chicken!"

The chicken took off running, but no one caught up with her. She found another bus, and as she boarded, the driver said, "Chicken!"

Again, the chicken took off running, but no one could catch her. She found another bus, and as she hopped onboard, someone shouted, "Chicken!"

Feeling embarrassed, the chicken took off running. She stumbled onto a train, and a passenger yelled, "Chicken!"

She jumped off the train before anyone else could say anything. The chicken was finally safe but realized she had become quite the commodity in urban transit!
 Why did the chicken join the dance group?
 Why did the chicken join a rock band? Because it had the right ingredients and wanted to get out of the "egg"!
 Why do chickens have such great memories? Because they eat so many eggs!
 Why did the chicken join t

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 Why do chickens make terrible comedians? Because they always stick to the basics without any punchlines!
 Why do chickens make terrible drivers?


In [6]:
#Using high top_p
for _ in range(5):
  output = pipe(messages, do_sample=True, top_p=1)
  print(output[0]['generated_text'])

 Why don't chickens use ATMs? Because they're afraid of roosters withdrawing too much "egg-ceptional" money!
 Why did the chicken join the comedy club? Because it had seen too many baked beans!
 Why did the chicken join a band?
 Why do chickens love playing hide and seek? Because good luck hiding when they're cooped up watching 'Chicken Run'!
 Why did the chicken go to the sewing class? Because it wanted to get over its pecking order!


In [7]:
#Using top_k (at most k tokens will consider at all tokens)
for _ in range(5):
  output = pipe(messages, do_sample=True, top_k=10)
  print(output[0]['generated_text'])

 Why did the chicken go to the bar? Because it couldn't hold it and needed a drink!
 Why did the chicken join the book club? Because it wanted to be a bit more 'egg-straordinary'!
 Why did the chicken join a rock band?


Becuase it had the perfect drumsticks!
 Why don't chickens play cards in the park? Because they're afraid of crows, and they already have enough pecking order in the house. Plus, the hens would just be too busy clucking about the chicken salad!
 Why don't chickens use smartphones? Because they're not good at texting, they'd never know how to say "cluck."
